Los conjuntos heterogéneos se pueden dividir en dos toipos, dependiendo de cómo combinen las predicciones individuales del estimador base en una predicción final:

* **Weighting methods**: Estos métodos asignan a las predicciones individuales del estimador base un peso que corresponde a su fuerza. A los mejores estimadores base se les asignan pesos más altos e influyen más en la predicción final general. Las predicciones de los estimadores de base individuales se introducen en una función de combinación predeterminada, que hace las predicciones finales.

* **Meta-learning methods**: Estos métodos usan un algoritmo de aprendizaje para combinar las predicciones de los estimadores base; las predicciones de los estimadores de base individuales se tratan como metadatos y se pasan a un meta-aprendiz de segundo nivel, que está capacitado para hacer predicciones finales.

Base estimators for heterogeneous ensembles

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split


X, y = make_moons(600, noise=0.25, random_state=13)

X, Xval, y, yval = train_test_split(X, y, test_size=0.25) # Sets aside 25% of the data for validation

Xtrn, Xtst, ytrn, ytst = train_test_split(X, y, test_size=0.25) # Sets aside a further 25% of the data for hold-out testing

Fitting different base estimators

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB


estimators = [
    ('dt', DecisionTreeClassifier (max_depth=5)),
    ('svm', SVC(gamma=1.0, C=1.0, probability=True)),
    ('gp', GaussianProcessClassifier(RBF(1.0))),
    ('3nn', KNeighborsClassifier(n_neighbors=3)),
    ('rf',RandomForestClassifier(max_depth=3, n_estimators=25)),
    ('gnb', GaussianNB())]

def fit(estimators, X, y):
    for model, estimator in estimators:
        # Fits base estimators on the training data using these different learning algorithms
        estimator.fit(X, y)
    return estimators

estimators = fit(estimators, Xtrn, ytrn)

Individual predictions of base estimators

In [ ]:
import numpy as np


# The flag “proba” allows us to predict labels or probability over the labels.
def predict_individual(X, estimators, proba=False):
    n_estimators = len(estimators)
    n_samples = X.shape[0]
    y = np.zeros((n_samples, n_estimators))
    for i, (model, estimator) in enumerate(estimators):
        if proba:
            # If true, predicts the probability of Class 1 (returns a float point probability value between 0 and 1)
            y[:, i] = estimator.predict_proba(X)[:, 1]
        else:
            # Otherwise, directly predicts Class 1 (returns an integer class label 0 or 1)
            y[:, i] = estimator.predict(X) 
    
    return y

## Majority Vote

In [ ]:
# Combining predictions using majority vote
from scipy.stats import mode


def combine_using_majority_vote(X, estimators):
    y_individual = predict_individual(X, estimators, proba=False)
    y_final = mode(y_individual, axis=1, keepdims=False)
    # Reshapes the vector to ensure it returns one prediction per example
    return y_final[0].reshape(-1, )

from sklearn.metrics import accuracy_score

ypred = combine_using_majority_vote(Xtst, estimators)
tst_err = 1 - accuracy_score(ytst, ypred)

Combining using **Accuracy Weighting**

In [ ]:
def combine_using_accuracy_weighting(X, estimators, Xval, yval): # Takes the validation set as input
    n_estimators = len(estimators)
    # Gets individual predictions on the validation set
    yval_individual = predict_individual(Xval, estimators, proba=False)
    # Sets the weight for each base classifier as its accuracy score
    wts = [accuracy_score(yval, yval_individual[:, i]) for i in range(n_estimators)]
    wts /= np.sum(wts) # Normalizes the weights
    ypred_individual = predict_individual(X, estimators, proba=False)
    y_final = np.dot(ypred_individual, wts) # Computes the weighted combination of individual labels efficiently
    return np.round(y_final) # Converts the combined prediction into a 0–1 label by rounding

ypred = combine_using_accuracy_weighting(Xtst, estimators, Xval, yval)
tst_err = 1 - accuracy_score(ytst, ypred)

## Entropy weighting

La entropía, o entropía de la información para ser precisos, fue originalmente ideada por Claude Shannon para cuantificar la “cantidad de información” transmitida por una variable. Esto está determinado por dos factores: (1) el número de valores distintos que puede tomar la variable y (2) la incertidumbre asociada con cada valor.

Considere que tres pacientes, Ana, Bob y Cam, están en el consultorio del médico esperando el diagnóstico de una enfermedad por parte del médico. A Ana se le dice con un 90 % de confianza que está sana (es decir, un 10 % de probabilidad de que esté enferma). A Bob se le dice con un 95 % de confianza que está enfermo (es decir, un 5 % de posibilidades de que esté sano). A Cam se le dice que los resultados de su prueba no son concluyentes (es decir, 50%/50%).
Ana ha recibido buenas noticias y hay poca incertidumbre en su diagnóstico. Aunque Bob ha recibido malas noticias, también hay poca incertidumbre en su diagnóstico.

La situación de Cam es de máxima incertidumbre: no ha recibido ni buenas ni malas noticias y le esperan más pruebas.
La entropía cuantifica esta noción de incertidumbre a través de varios resultados. Las medidas basadas en la entropía se usan comúnmente durante el aprendizaje del árbol de decisiones para identificar con avidez las mejores variables para dividir y se usan como funciones de pérdida en redes neuronales profundas.

In [ ]:
def entropy(y):
    _, counts = np.unique(y, return_counts=True)
    p = np.array(counts.astype('float') / len(y))
    ent = -p.T @ np.log2(p)
    return ent

Hay dos diferencias clave entre la ponderación de entropía y la ponderación de precisión:

* La precisión de un clasificador base se calcula usando las etiquetas verdaderas ytrue y las etiquetas predichas ypred. De esta manera, la métrica de precisión mide qué tan bien se desempeña un clasificador. Un clasificador con alta precisión es mejor.
* La entropía de un clasificador base se calcula utilizando solo las etiquetas predichas ypred, y la métrica de entropía mide la incertidumbre de un clasificador sobre sus predicciones. Un clasificador con baja entropía (incertidumbre) es mejor. Por lo tanto, los pesos de los clasificadores base individuales son inversamente proporcionales a sus correspondientes entropías.

Combining using entropy weighting

In [ ]:
def combine_using_entropy_weighting(X, estimators, Xval): # Takes only the validation examples
    n_estimators = len(estimators)
    # Gets individual predictions on the validation set
    yval_individual = predict_individual(Xval, estimators, proba=False) 
    # Sets the weight for each base classifier as its inverse entropy
    wts = [1/entropy(yval_individual[:, i]) for i in range(n_estimators)]
    wts /= np.sum(wts) # Normalizes the weights
    ypred_individual = predict_individual(X, estimators, proba=False)
    y_final = np.dot(ypred_individual, wts) # Computes the weighted combination of individual labels efficiently
    return np.round(y_final) # Returns the rounded predictions

ypred = combine_using_entropy_weighting(Xtst, estimators, Xval)
tst_err = 1 - accuracy_score(ytst, ypred)

## Dempster-Shafer combination

In [ ]:
def combine_using_Dempster_Schafer(X, estimators):
    p_individual = predict_individual(X, estimators, proba=True) # Gets individual predictions on the validation set
    bpa0 = 1.0 - np.prod(p_individual, axis=1)
    bpa1 = 1.0 - np.prod(1 - p_individual, axis=1)
    # Stacks the beliefs for Class 0 and Class 1 side by side for every test example
    belief = np.vstack([bpa0 / (1 - bpa0), bpa1 / (1 - bpa1)]).T
    y_final = np.argmax(belief, axis=1) # Selects the final label as the class with the highest belief
    return y_final

ypred = combine_using_Dempster_Schafer(Xtst, estimators)
tst_err = 1 - accuracy_score(ytst, ypred)

## Combining predictions by meta-learning

### Stacking
El apilamiento es el método de meta-aprendizaje más común y recibe su nombre porque
apila un segundo clasificador encima de sus estimadores base. El procedimiento general de apilamiento
tiene dos pasos:
* Nivel 1: Ajustar estimadores base sobre los datos de entrenamiento. Este paso es el mismo que antes.
y tiene como objetivo crear un conjunto diverso y heterogéneo de clasificadores básicos.

* Nivel 2: Construir un nuevo conjunto de datos a partir de las predicciones de los clasificadores base,
que se convierten en meta-características. Las meta-características pueden ser las predicciones o las
probabilidad de predicciones.

<img src="img/stacking.jpg">

*Nota: El estimador de nivel 2 aquí se puede entrenar usando cualquier algoritmo de aprendizaje. historicamente, se han utilizado modelos lineales como la regresión lineal y la regresión logística.*

Stacking with a second-level estimator

In [ ]:
def fit_stacking(level1_estimators, level2_estimator, X, y, use_probabilities=False):
    fit(level1_estimators, X, y) # Trains level-1 base estimators
    # Gets meta-features as individual predictions or prediction probabilities (proba=True/False)
    X_meta = predict_individual(X, estimators=level1_estimators, proba=use_probabilities)
    level2_estimator.fit(X_meta, y)
    final_model = {'level-1': level1_estimators,
                   'level-2': level2_estimator, # Saves the level-1 estimators and level-2 estimator in a dictionary
                   'use-proba': use_probabilities}
    return final_model

Making predictions with a stacked model

In [ ]:
def predict_stacking(X, stacked_model):
    level1_estimators = stacked_model['level-1'] # Gets level-1 base estimators
    use_probabilities = stacked_model['use-proba']
    # Gets meta-features using the level-1 base estimators
    X_meta = predict_individual(X, estimators=level1_estimators, proba=use_probabilities)
    level2_estimator = stacked_model['level-2']
    # Gets level-2 estimator and uses it to make the final predictions on the meta-features
    y = level2_estimator.predict(X_meta) 
    return y

In [ ]:
# Example

from sklearn.linear_model import LogisticRegression


meta_estimator = LogisticRegression(C=1.0, solver='lbfgs')

stacking_model = fit_stacking(estimators, meta_estimator, Xtrn, ytrn, use_probabilities=True)

ypred = predict_stacking(Xtst, stacking_model)

tst_err = 1 - accuracy_score(ytst, ypred)

*Nota: El Stacking (Apilamiento) genera overfitting en los resultados, se puede mejorar esto utilizando k-fold cross validation en el entrenamiento*

Stacking with cross validation

In [ ]:
from sklearn.model_selection import StratifiedKFold


def fit_stacking_with_CV(level1_estimators, level2_estimator, X, y, n_folds=5, use_probabilities=False):
    n_samples = X.shape[0]
    n_estimators = len(level1_estimators)
    X_meta = np.zeros((n_samples, n_estimators)) # Initializes the metadata matrix
    splitter = StratifiedKFold(n_splits=n_folds, shuffle=True)
    # Trains level-1 estimators and then makes meta-features for the level-2 estimator with individual predictions
    for trn, val in splitter.split(X, y):
        level1_estimators = fit(level1_estimators, X[trn, :], y[trn])
        X_meta[val, :] = predict_individual(X[val, :], estimators=level1_estimators, proba=use_probabilities)
    level2_estimator.fit(X_meta, y)
    level1_estimators = fit(level1_estimators, X, y)
    final_model = {'level-1': level1_estimators,
                   'level-2': level2_estimator, # Saves the level-1 estimators and level-2 estimator in a dictionary
                   'use-proba': use_probabilities} 
    return final_model

In [ ]:
# Example

stacking_model = fit_stacking_with_CV(estimators, 
                                      meta_estimator, 
                                      Xtrn, 
                                      ytrn, 
                                      n_folds=5, 
                                      use_probabilities=True)

ypred = predict_stacking(Xtst, stacking_model)

tst_err = 1 - accuracy_score(ytst, ypred)